In [11]:
import string
import xml.etree.ElementTree as ET
from collections import OrderedDict, defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [12]:
%load_ext tensorboard

In [13]:
#Import dataset
tree = ET.parse('lag1734.xml/lag1734.xml')
root = tree.getroot()
root.attrib

{'id': 'lag1734'}

In [14]:
# Import stopwords
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f.readlines()]
stop_words

['.',
 ',',
 '?',
 ';',
 'cap',
 'och',
 'eller',
 'som',
 'thet',
 'ej',
 'han',
 'then',
 'ther',
 'til',
 'om',
 'at',
 'med',
 'tå',
 'för',
 'the',
 'af']

In [15]:
chapters = []
for child in root:
    chapters.append(child)

sentances = []
for chapter in chapters:
    for paragraph in chapter:
        for sentence in paragraph:
            sentance = ''
            for child in sentence:
                sentance += f'{child.text} '
            sentances.append(sentance[:-1])
sentances

['D O M A R E R E G L E R. Någre almennelige Regler , ther en Domare skal sigh aldeles effter rätta .',
 'En Domare skal först besinna , at han en Gudz Befalningsman , och thet Embete han förer , thet hörer Gudh til , och icke honom sielffuom , och therföre hörer Domen , som han afsäger , Gudhi til , efter thet han afsagd warder i Gudz Embete på Gudz wegna , så at thet är wisserliga Gudz Dom , och icke Menniskiors .',
 'Och ty ligger Domaren ther Macht vppå , at han seer sigh wijsligen före , at han icke på Gudz wegna dömer en falskan Dom , med hwilken han dömer sig til en ewigh Fördömelse , effter thet han misbrukat Guds Dom och Befalning til Öffuerwold och Orätt , som til Rätt af Gudhi insatt är .',
 'Men ther han haffuer wilia til at döma Rätt , och ransakar grant effter sitt ytersta Förstånd om Rätten , och kan dock icke för sin Oförståndigheet finna på Rätten , och säger så en falsk Dom , tå haffuer han någor Vrsächt , at han är kommen på then falska Domen , emot sin wilia aff wåd

In [16]:
#Pre-processing

In [17]:
#Tokenization
tfidf = TfidfVectorizer(stop_words = stop_words)
X = tfidf.fit_transform(sentances)
print(*tfidf.get_feature_names_out())

10 100 11 113 12 128 13 133 14 1442 14de 15 16 1608 1618 1686 17 1731 1734 1736 18 19 20 21 22 23 235 24 247 25 256 26 27 28 289 29 30 308 31 32 33 34 342 35 36 37 38 39 40 41 42 423 47 53 60 600 700 87 acht achtandes achte adelig aderton adertonde adolph afbedje afbida afbidas afbide afbrändes afbärgad afbön afdelta afdragas afdömd aff affall affhender affkomma afflytta afflytter afflyttning affsagda affsagdt affsked affstraffas affsägas affund affwundz affwundzman affwäria affödo afgift afgiften afgiord afgiorda afgiort afgiöra afgiöras afguda afgäld afgäldet afgärda afgår afhandlas afhandling afhugg afhuggne afhyst afhyste afhänder afhändt afhöra afhördt afhöre afkomma afkommo afkomst afla aflad aflade afladt aflar aflas afledna afleggias aflinge aflingejord aflingejorden aflop aflops aflopsdike aflyft afrad afraden afradet afradsdag afräkas afräkna afrödje afrögdt afsagd afsagdt afsalu afseende afsides afsked afskedspass afskied afskild afskildt afskrifna afskrift afskrifter afskri

In [18]:
feature_array = np.array(tfidf.get_feature_names_out())
tfidf_sorting = np.argsort(X.toarray().sum(axis=0))[::-1]
n = 100
top_n = feature_array[tfidf_sorting[:n]]
print(top_n)

['är' 'böte' 'man' 'daler' 'må' 'så' 'sagdt' 'någor' 'sig' 'ock' 'skal'
 '10' 'sin' 'honom' 'efter' 'annan' 'utan' 'sedan' 'vare' 'them' 'samma'
 '11' 'än' 'en' 'hafver' 'rätt' 'äro' 'domaren' 'nu' 'giör' 'kan' 'åter'
 'konungens' 'lag' 'sitt' 'sägs' 'hans' 'hafve' 'vid' 'saken' 'förr' '12'
 'thes' 'gånge' '13' 'gods' 'vil' 'sker' 'skada' 'hvar' 'huru' 'bör' 'tid'
 'hus' 'på' 'rätten' 'kommer' 'hofrätten' 'emot' 'in' 'laga' 'annat'
 'andra' 'annars' 'tijo' 'balken' 'något' 'ware' 'äntå' 'staden' 'skadan'
 'ifrån' 'jord' 'sielf' 'under' 'alt' 'någon' 'skola' 'äger' 'barn'
 'landet' 'thertil' 'vara' 'inom' 'niute' '14' 'plichte' 'ingen'
 'konungen' 'hvad' 'gifve' 'gälde' 'dom' 'stadgadt' 'hafva' 'tage'
 'mindre' 'finnes' 'dag' 'ut']


In [19]:
feature_array = np.array(tfidf.get_feature_names_out())
tfidf_sorting = np.argsort(X.toarray().sum(axis=0))
n = 1000
top_n = feature_array[tfidf_sorting[:n]]
print(top_n)

['jämteland' 'lagmansdomen' 'småland' 'södermanland' 'skaraborgs'
 'tavastehus' 'medelpad' 'ångermanland' 'göta' 'kopparbergs' 'bohuslän'
 'ingifve' 'halland' 'helsingeland' 'dahl' 'herjedalen' 'västmanland'
 'västerbotn' 'elfsborgs' 'nyland' 'upland' 'nerike' 'vermeland' 'kalmare'
 'kymmenegårds' 'österbotn' 'rautalambi' 'gothland' 'gestrikeland'
 'östergöthland' 'blekinge' 'skåne' 'angelägit' 'förändringar' 'adolph'
 'ehuruväl' 'förändrat' 'gångne' 'nöigt' 'sorgfällighet' 'svårigheter'
 'öfversedde' 'lagfarne' 'riksdag' 'gustaf' 'brukelig' 'utesluta'
 'brukliga' 'bringas' 'berömmelig' 'vittre' 'behöringen' 'svänska'
 'förständiga' 'önskan' 'mångahanda' 'utgifvande' 'stadslagens' 'sedvana'
 'härtils' 'ändskap' 'förbättra' 'förbättrade' 'högloflige' 'hälsosamt'
 '1731' 'esomoftast' 'högtärade' 'företrädare' '1618' 'lärda' 'ändrade'
 'emellankomna' 'våre' 'daga' 'samtelige' 'sinnande' 'moederfaders'
 'tiänte' '700' 'enskylt' 'efterlevande' 'landslag' 'förändrades' 'allmän'
 'förenad' 'g

In [20]:
counter = CountVectorizer(stop_words=stop_words)
Y = counter.fit_transform(sentances)
Y_sum = Y.toarray().sum(axis=0)
bpe_words = list(zip(map(lambda x: ' '.join(list(x)) + ' </w>', counter.get_feature_names_out()), Y_sum))
bpe_words

[('1 0 </w>', 58),
 ('1 0 0 </w>', 2),
 ('1 1 </w>', 40),
 ('1 1 3 </w>', 1),
 ('1 2 </w>', 29),
 ('1 2 8 </w>', 1),
 ('1 3 </w>', 35),
 ('1 3 3 </w>', 1),
 ('1 4 </w>', 23),
 ('1 4 4 2 </w>', 1),
 ('1 4 d e </w>', 1),
 ('1 5 </w>', 16),
 ('1 6 </w>', 21),
 ('1 6 0 8 </w>', 2),
 ('1 6 1 8 </w>', 1),
 ('1 6 8 6 </w>', 1),
 ('1 7 </w>', 12),
 ('1 7 3 1 </w>', 1),
 ('1 7 3 4 </w>', 3),
 ('1 7 3 6 </w>', 1),
 ('1 8 </w>', 11),
 ('1 9 </w>', 6),
 ('2 0 </w>', 6),
 ('2 1 </w>', 7),
 ('2 2 </w>', 8),
 ('2 3 </w>', 7),
 ('2 3 5 </w>', 1),
 ('2 4 </w>', 5),
 ('2 4 7 </w>', 1),
 ('2 5 </w>', 11),
 ('2 5 6 </w>', 1),
 ('2 6 </w>', 8),
 ('2 7 </w>', 8),
 ('2 8 </w>', 3),
 ('2 8 9 </w>', 1),
 ('2 9 </w>', 3),
 ('3 0 </w>', 8),
 ('3 0 8 </w>', 1),
 ('3 1 </w>', 3),
 ('3 2 </w>', 4),
 ('3 3 </w>', 2),
 ('3 4 </w>', 3),
 ('3 4 2 </w>', 1),
 ('3 5 </w>', 2),
 ('3 6 </w>', 3),
 ('3 7 </w>', 3),
 ('3 8 </w>', 1),
 ('3 9 </w>', 1),
 ('4 0 </w>', 1),
 ('4 1 </w>', 1),
 ('4 2 </w>', 3),
 ('4 2 3 </w>', 1),


In [21]:
def get_pair_stats(vocab: 'list[tuple[str, int]]'):
    pairs: 'dict[tuple[str], int]' = {}
    for word, frequency in vocab:
        symbols = word.split()

        # count occurrences of pairs
        for i in range(len(symbols) - 1):
            pair = (symbols[i], symbols[i + 1])
            current_frequency = pairs.get(pair, 0)
            pairs[pair] = current_frequency + frequency

    return pairs

pairs = get_pair_stats(bpe_words)
pairs

{('1', '0'): 60,
 ('0', '</w>'): 78,
 ('0', '0'): 4,
 ('1', '1'): 41,
 ('1', '</w>'): 52,
 ('1', '3'): 37,
 ('3', '</w>'): 48,
 ('1', '2'): 30,
 ('2', '</w>'): 46,
 ('2', '8'): 5,
 ('8', '</w>'): 20,
 ('3', '3'): 3,
 ('1', '4'): 25,
 ('4', '</w>'): 34,
 ('4', '4'): 1,
 ('4', '2'): 6,
 ('4', 'd'): 1,
 ('d', 'e'): 3829,
 ('e', '</w>'): 6475,
 ('1', '5'): 16,
 ('5', '</w>'): 30,
 ('1', '6'): 25,
 ('6', '</w>'): 35,
 ('6', '0'): 4,
 ('0', '8'): 3,
 ('6', '1'): 1,
 ('1', '8'): 12,
 ('6', '8'): 1,
 ('8', '6'): 1,
 ('1', '7'): 17,
 ('7', '</w>'): 26,
 ('7', '3'): 5,
 ('3', '1'): 4,
 ('3', '4'): 7,
 ('3', '6'): 4,
 ('1', '9'): 6,
 ('9', '</w>'): 11,
 ('2', '0'): 6,
 ('2', '1'): 7,
 ('2', '2'): 8,
 ('2', '3'): 9,
 ('3', '5'): 3,
 ('2', '4'): 6,
 ('4', '7'): 2,
 ('2', '5'): 12,
 ('5', '6'): 1,
 ('2', '6'): 8,
 ('2', '7'): 8,
 ('8', '9'): 1,
 ('2', '9'): 3,
 ('3', '0'): 9,
 ('3', '2'): 4,
 ('3', '7'): 3,
 ('3', '8'): 1,
 ('3', '9'): 1,
 ('4', '0'): 1,
 ('4', '1'): 1,
 ('5', '3'): 1,
 ('7', '0'): 

In [22]:
import re

def merge_vocab(best_pair: 'tuple[str, str]', vocab_in: 'list[tuple[str, int]]'):

    vocab_out: 'dict[str, int]' = {}

    # re.escape
    # ensures the characters of our input pair will be handled as is and
    # not get mistreated as special characters in the regular expression.
    pattern = f'(^| ){re.escape(" ".join(best_pair))}($| )'
    replacement = r'\g<1>' + ''.join(best_pair) + r'\g<2>'

    for word_in, freq in vocab_in:
        # replace most frequent pair in all vocabulary
        word_out = re.sub(pattern, replacement, word_in)
        vocab_out[word_out] = freq

    return [(word, frequency) for word, frequency in vocab_out.items()]

In [23]:
best_pair = max(pairs, key=pairs.get)
print(best_pair)

new_vocab = merge_vocab(best_pair, bpe_words)
new_vocab

('r', '</w>')


[('1 0 </w>', 58),
 ('1 0 0 </w>', 2),
 ('1 1 </w>', 40),
 ('1 1 3 </w>', 1),
 ('1 2 </w>', 29),
 ('1 2 8 </w>', 1),
 ('1 3 </w>', 35),
 ('1 3 3 </w>', 1),
 ('1 4 </w>', 23),
 ('1 4 4 2 </w>', 1),
 ('1 4 d e </w>', 1),
 ('1 5 </w>', 16),
 ('1 6 </w>', 21),
 ('1 6 0 8 </w>', 2),
 ('1 6 1 8 </w>', 1),
 ('1 6 8 6 </w>', 1),
 ('1 7 </w>', 12),
 ('1 7 3 1 </w>', 1),
 ('1 7 3 4 </w>', 3),
 ('1 7 3 6 </w>', 1),
 ('1 8 </w>', 11),
 ('1 9 </w>', 6),
 ('2 0 </w>', 6),
 ('2 1 </w>', 7),
 ('2 2 </w>', 8),
 ('2 3 </w>', 7),
 ('2 3 5 </w>', 1),
 ('2 4 </w>', 5),
 ('2 4 7 </w>', 1),
 ('2 5 </w>', 11),
 ('2 5 6 </w>', 1),
 ('2 6 </w>', 8),
 ('2 7 </w>', 8),
 ('2 8 </w>', 3),
 ('2 8 9 </w>', 1),
 ('2 9 </w>', 3),
 ('3 0 </w>', 8),
 ('3 0 8 </w>', 1),
 ('3 1 </w>', 3),
 ('3 2 </w>', 4),
 ('3 3 </w>', 2),
 ('3 4 </w>', 3),
 ('3 4 2 </w>', 1),
 ('3 5 </w>', 2),
 ('3 6 </w>', 3),
 ('3 7 </w>', 3),
 ('3 8 </w>', 1),
 ('3 9 </w>', 1),
 ('4 0 </w>', 1),
 ('4 1 </w>', 1),
 ('4 2 </w>', 3),
 ('4 2 3 </w>', 1),


In [24]:
bpe_codes = OrderedDict()
num_merges = 1000  # hyperparameter
vocab = bpe_words
for i in range(num_merges):
    print('\niteration', i)
    pair_stats = get_pair_stats(vocab)
    if not pair_stats:
        break

    best_pair = max(pair_stats, key=pair_stats.get)
    bpe_codes[best_pair] = i

    # print('vocabulary: ', vocab)
    print('best pair:', best_pair)
    vocab = merge_vocab(best_pair, vocab)

print('\nfinal vocabulary: ', vocab)
print('\nbyte pair encoding: ', bpe_codes)


iteration 0
best pair: ('r', '</w>')

iteration 1
best pair: ('n', '</w>')

iteration 2
best pair: ('e', '</w>')

iteration 3
best pair: ('t', '</w>')

iteration 4
best pair: ('a', '</w>')

iteration 5
best pair: ('s', '</w>')

iteration 6
best pair: ('e', 'r</w>')

iteration 7
best pair: ('e', 'n</w>')

iteration 8
best pair: ('a', 'r')

iteration 9
best pair: ('a', 'n')

iteration 10
best pair: ('s', 't')

iteration 11
best pair: ('d', '</w>')

iteration 12
best pair: ('s', 'k')

iteration 13
best pair: ('n', 'g')

iteration 14
best pair: ('a', 'n</w>')

iteration 15
best pair: ('e', 'r')

iteration 16
best pair: ('a', 'l')

iteration 17
best pair: ('ö', 'r')

iteration 18
best pair: ('o', 'm')

iteration 19
best pair: ('a', 'g')

iteration 20
best pair: ('f', 'v')

iteration 21
best pair: ('e', 'n')

iteration 22
best pair: ('i', 'g')

iteration 23
best pair: ('r', 'ä')

iteration 24
best pair: ('f', 'ör')

iteration 25
best pair: ('t', 't')

iteration 26
best pair: ('a', 'r</w>')


In [25]:
bpe_codes

OrderedDict([(('r', '</w>'), 0),
             (('n', '</w>'), 1),
             (('e', '</w>'), 2),
             (('t', '</w>'), 3),
             (('a', '</w>'), 4),
             (('s', '</w>'), 5),
             (('e', 'r</w>'), 6),
             (('e', 'n</w>'), 7),
             (('a', 'r'), 8),
             (('a', 'n'), 9),
             (('s', 't'), 10),
             (('d', '</w>'), 11),
             (('s', 'k'), 12),
             (('n', 'g'), 13),
             (('a', 'n</w>'), 14),
             (('e', 'r'), 15),
             (('a', 'l'), 16),
             (('ö', 'r'), 17),
             (('o', 'm'), 18),
             (('a', 'g'), 19),
             (('f', 'v'), 20),
             (('e', 'n'), 21),
             (('i', 'g'), 22),
             (('r', 'ä'), 23),
             (('f', 'ör'), 24),
             (('t', 't'), 25),
             (('a', 'r</w>'), 26),
             (('ä', 'r</w>'), 27),
             (('a', 'd'), 28),
             (('t', 'h'), 29),
             (('i', 'ng'), 30),
      

In [26]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [27]:
single_char_vocab = list(set((string.printable[:-5]+ 'åäö').lower())) + ['</w>']
tokenizer_dict = defaultdict(lambda: 1)
tokenizer_dict['<pad>'] = 0
for idx, char in enumerate(single_char_vocab):
    tokenizer_dict[char] = idx+2
tokenizer_dict.update({''.join(word): idx+len(tokenizer_dict)+2 for word, idx in bpe_codes.items()})
# tokenizer_list
tokenizer_dict


defaultdict(<function __main__.<lambda>()>,
            {'<pad>': 0,
             'w': 2,
             'r': 3,
             "'": 4,
             'p': 5,
             'm': 6,
             'q': 7,
             '1': 8,
             '$': 9,
             '|': 10,
             ')': 11,
             ':': 12,
             '\\': 13,
             'd': 14,
             '&': 15,
             '~': 16,
             't': 17,
             'x': 18,
             '*': 19,
             '8': 20,
             'v': 21,
             'k': 22,
             ' ': 23,
             '=': 24,
             'y': 25,
             'ä': 26,
             '!': 27,
             '?': 28,
             'ö': 29,
             's': 30,
             '#': 31,
             'i': 32,
             '4': 33,
             '9': 34,
             '{': 35,
             '7': 36,
             'g': 37,
             ']': 38,
             '/': 39,
             '6': 40,
             'j': 41,
             '}': 42,
             '-': 43,
             '

## Regex speedup test

In [50]:
class Trie():
    #  https://stackoverflow.com/a/42789508
    """Regex::Trie in Python. Creates a Trie out of a list of words. The trie can be exported to a Regex pattern.
    The corresponding Regex should match much faster than a simple Regex union."""

    def __init__(self):
        self.data = {}

    def add(self, word):
        ref = self.data
        for char in word:
            ref[char] = char in ref and ref[char] or {}
            ref = ref[char]
        ref[''] = 1

    def dump(self):
        return self.data

    def quote(self, char):
        return re.escape(char)

    def _pattern(self, pData):
        data = pData
        if "" in data and len(data.keys()) == 1:
            return None

        alt = []
        cc = []
        q = 0
        for char in data.keys():
            if isinstance(data[char], dict):
                try:
                    recurse = self._pattern(data[char])
                    alt.append(self.quote(char) + recurse)
                except:
                    cc.append(self.quote(char))
            else:
                q = 1
        cconly = not len(alt) > 0

        if len(cc) > 0:
            if len(cc) == 1:
                alt.append(cc[0])
            else:
                alt.append('[' + ''.join(cc) + ']')

        if len(alt) == 1:
            result = alt[0]
        else:
            result = "(?:" + "|".join(alt) + ")"

        if q:
            if cconly:
                result += "?"
            else:
                result = "(?:%s)?" % result
        return result

    def pattern(self):
        return self._pattern(self.dump())

In [51]:
trie = Trie()
for pair in bpe_codes:
    trie.add(" ".join(pair))
bpe_regex_pattern = re.compile(r"(?<=\b)(" + trie.pattern() + r")(?=\b)", re.IGNORECASE)
bpe_regex_pattern

re.compile(r'(\b)((?:r(?:\ (?:</w>|e(?:</w>|d)|a(?:</w>|s</w>|r</w>|n)|[äiåy])|ä(?:\ (?:tt(?:</w>)?|[nk])|tt(?:\ (?:e(?:(?:n(?:</w>|s</w>)|</w>))?|a</w>)|e\ gång)|n\ ta</w>)|å(?:\ d(?:</w>)?|d(?:\ stu|stu(?:\ fv|fv(?:\ u|u\ rätt</w>))))|i\ (?:s</w>|k)|an\ sak)|n(?:\ (?:</w>|d(?:</w>)?|i(?:(?:ng|u))?|a(?:(?:</w>|d(?:</w>)?|r</w>|s</w>))?|e(?:</w>|n</w>|s</w>|t</w>)|u</w>|ä(?:(?:r(?:</w>)?|m))?|öd?|[gåy])|å(?:\ go|go\ (?:r</w>|t</w>|n</w>))|i(?:ng\ (?:</w>|ar</w>|en</w>|s</w>)|u\ te</w>)|d\ (?:e(?:</w>|r</w>)|r)|y\ tt|ä(?:\ sta</w>|m\ nd)|g\ </w>)|e(?:\ (?:</w>|r(?:</w>)?|n(?:</w>)?|s(?:</w>)?|t(?:(?:</w>|t</w>))?|d(?:</w>)?|g(?:en)?|hvad</w>|[lmf])|n\ (?:s</w>|thera</w>|de</w>)|m\ (?:</w>|ot</w>|ellan</w>|an</w>|bet)|f\ ter</w>|d\ (?:an</w>|er(?:</w>)?)|l(?:\ (?:</w>|ig(?:a</w>|</w>)|s(?:e</w>)?|je|t</w>|l)|l\ an</w>|je\ st</w>)|r\ (?:a</w>|s(?:</w>)?)|t\ s</w>|gen\ dom</w>)|t(?:\ (?:</w>|t(?:</w>)?|i(?:ng(?:</w>)?)?|e(?:(?:</w>|r(?:</w>)?|n</w>|liga</w>|s</w>))?|a(?:(?:</w>|[gl]))?|r(?

In [67]:
def sub_string(match: 're.Match'):
    return match.group(2).replace(' ','')
    

def tokenize(sentence: str, bpe_pattern: 're.Pattern'):
    sentence = sentence.lower()
    words = [' '.join(list(word)) for word in sentence.split()]
    for idx, word in enumerate(words):
        # replace most frequent pair in all vocabulary
        word_out = re.sub(bpe_pattern, sub_string, word)
        words[idx] = word_out
    tokens = []
    for idx, word in enumerate(words):
        bpe_tokens = word.split()
        tokenization = [tokenizer_dict[token] for token in bpe_tokens]
        tokens += tokenization
    return tokens

def tokenize_corpus(corpus: 'list[str]', bpe_codes: 'OrderedDict[tuple[str, str], int]'):
    tokens = []
    for i, sentence in enumerate(corpus):
        print(i)
        tokens.append(tokenize(sentence, bpe_codes))
    return tokens

tokenized_corpus = tokenize_corpus(sentances, bpe_regex_pattern) 
print(*tokenized_corpus, sep='\n')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## End of test

In [48]:
def tokenize(sentence: str, bpe: 'OrderedDict[tuple[str, str], int]'):
    sentence = sentence.lower()
    words = [' '.join(list(word)) for word in sentence.split()]
    for pair in bpe:
        pattern = f'(^| ){re.escape(" ".join(pair))}($| )'
        replacement = r'\g<1>' + ''.join(pair) + r'\g<2>'
        for idx, word in enumerate(words):
            # replace most frequent pair in all vocabulary
            word_out = re.sub(pattern, replacement, word)
            words[idx] = word_out
    tokens = []
    for idx, word in enumerate(words):
        bpe_tokens = word.split()
        tokenization = [tokenizer_dict[token] for token in bpe_tokens]
        tokens += tokenization
    return tokens

def tokenize_corpus(corpus: 'list[str]', bpe_codes: 'OrderedDict[tuple[str, str], int]'):
    tokens = []
    for i, sentence in enumerate(corpus):
        print(i)
        tokens.append(tokenize(sentence, bpe_codes))
    return tokens

tokenized_corpus = tokenize_corpus(sentances[1:10], bpe_codes) 
print(*tokenized_corpus, sep='\n')

0
1
2
3
4
5
6
7
8
[97, 173, 84, 69, 195, 100, 86, 138, 121, 55, 812, 59, 431, 47, 85, 97, 778, 53, 462, 583, 59, 66, 159, 105, 708, 116, 138, 413, 47, 85, 100, 91, 59, 105, 708, 683, 91, 778, 47, 327, 59, 66, 159, 32, 133, 69, 47, 137, 94, 364, 510, 94, 59, 66, 159, 145, 490, 683, 91, 173, 97, 59, 30, 94, 47, 85, 176, 306, 91, 59, 778, 351, 327, 59, 179, 315, 105, 708, 47, 85, 176, 267, 14, 245, 287, 32, 778, 53, 116, 138, 413, 997, 778, 53, 2, 770, 812, 59, 208, 431, 105, 708, 110, 206, 30, 30, 91, 397, 48, 778, 53, 173, 59, 66, 159, 32, 133, 69, 604, 713, 156, 113, 30, 44]
[66, 159, 632, 397, 1026, 173, 84, 97, 145, 378, 320, 21, 503, 73, 59, 431, 47, 85, 434, 91, 30, 98, 47, 206, 41, 367, 98, 97, 490, 59, 431, 47, 85, 32, 133, 69, 997, 778, 53, 2, 770, 812, 392, 91, 97, 190, 88, 85, 173, 59, 540, 47, 666, 22, 97, 47, 85, 392, 91, 30, 98, 327, 97, 69, 2, 98, 47, 100, 392, 108, 434, 59, 179, 243, 91, 105, 708, 47, 85, 479, 427, 22, 431, 778, 30, 173, 66, 159, 384, 327, 29, 510, 91, 2,

# Embedding

In [72]:
inverse_vocab = {index: token for token, index in tokenizer_dict.items()}

In [80]:
window_size = 2
vocab_size = len(tokenizer_dict)
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      tokenized_corpus[1],
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
positive_skip_grams

[[36, 708],
 [36, 327],
 [72, 18],
 [67, 94],
 [133, 159],
 [85, 67],
 [176, 85],
 [778, 59],
 [812, 121],
 [267, 63],
 [59, 26],
 [66, 72],
 [778, 72],
 [778, 72],
 [36, 812],
 [351, 327],
 [490, 159],
 [36, 43],
 [812, 36],
 [72, 66],
 [364, 94],
 [91, 85],
 [36, 173],
 [85, 267],
 [43, 156],
 [43, 91],
 [36, 67],
 [708, 116],
 [778, 327],
 [36, 431],
 [364, 137],
 [67, 413],
 [36, 91],
 [86, 138],
 [176, 63],
 [138, 86],
 [179, 105],
 [133, 67],
 [100, 67],
 [85, 67],
 [18, 778],
 [26, 145],
 [67, 137],
 [778, 18],
 [159, 36],
 [36, 351],
 [86, 100],
 [778, 351],
 [97, 72],
 [245, 287],
 [327, 36],
 [18, 397],
 [72, 997],
 [36, 778],
 [91, 36],
 [91, 36],
 [138, 72],
 [173, 778],
 [91, 43],
 [267, 85],
 [413, 778],
 [364, 94],
 [306, 36],
 [67, 36],
 [778, 683],
 [138, 121],
 [708, 105],
 [59, 159],
 [113, 58],
 [67, 138],
 [66, 770],
 [59, 245],
 [97, 778],
 [43, 67],
 [778, 66],
 [94, 364],
 [327, 179],
 [26, 36],
 [195, 100],
 [176, 306],
 [91, 778],
 [94, 67],
 [116, 138],
 [63,

In [77]:
print([inverse_vocab[t] for t in tokenized_corpus[1]])

['en', 'dom', 'ar', 'e', 'skal', 'för', 'st', 'be', 'si', 'n', 'na', ',', 'at', 'h', 'an', 'en', 'gud', 'z', 'befalnings', 'man', ',', 'o', 'ch', 'th', 'et', 'em', 'be', 'te', 'h', 'an', 'för', 'er', ',', 'th', 'et', 'hör', 'er', 'gud', 'h', 'til', ',', 'o', 'ch', 'i', 'ck', 'e', 'h', 'on', 'om', 'siel', 'ffu', 'om', ',', 'o', 'ch', 'ther', 'före', 'hör', 'er', 'dom', 'en', ',', 's', 'om', 'h', 'an', 'af', 'säg', 'er', ',', 'gud', 'hi', 'til', ',', 'ef', 'ter', 'th', 'et', 'h', 'an', 'af', 'sag', 'd', 'war', 'der', 'i', 'gud', 'z', 'em', 'be', 'te', 'på', 'gud', 'z', 'w', 'eg', 'na', ',', 'så', 'at', 'th', 'et', 'är', 'wi', 's', 's', 'er', 'lig', 'a', 'gud', 'z', 'dom', ',', 'o', 'ch', 'i', 'ck', 'e', 'men', 'ni', 'ski', 'or', 's', '.']


In [79]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(397, 778): (lig, gud)
(812, 431): (na, at)
(351, 36): (hi, ,)
(105, 179): (th, ef)
(116, 708): (em, et)


In [86]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    # seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([45  1  2 49], shape=(4,), dtype=int64)
['/', '–', '8', 'j']


In [87]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

In [88]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 36
target_word     : ,
context_indices : [708  45   1   2  49]
context_words   : ['et', '/', '–', '8', 'j']
label           : [1 0 0 0 0]


In [89]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : 36
context : tf.Tensor([708  45   1   2  49], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [90]:
import tqdm
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [113]:
targets, contexts, labels = generate_training_data(tokenized_corpus, window_size=2, num_ns=4, vocab_size=vocab_size, seed=42)
targets, contexts, labels = np.array(targets), np.array(contexts), np.array(labels)
print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 10/10 [00:00<00:00, 109.89it/s]



targets.shape: (310,)
contexts.shape: (310, 5)
labels.shape: (310, 5)


In [157]:
BATCH_SIZE = 32
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(32,), dtype=tf.int32, name=None), TensorSpec(shape=(32, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(32, 5), dtype=tf.int64, name=None))>


In [158]:
list(dataset)

[((<tf.Tensor: shape=(32,), dtype=int32, numpy=
   array([ 67, 293,  59, 378, 173, 159,  11, 708, 243, 145, 104, 708, 159,
          708,  67, 133,  51,  91, 587,  72, 378,  92, 267, 327, 105,  51,
          587, 997, 243, 320, 997, 812])>,
   <tf.Tensor: shape=(32, 5), dtype=int64, numpy=
   array([[  85,    7,    1,   30,   51],
          [ 145,   19,  576,  456,  177],
          [  34,   10,  199,   78,    1],
          [  34,    4,   11,  165,    9],
          [  34,   12,   16,  262,   73],
          [ 212,    7,    3,   50,   22],
          [  98,  184,    3,  385,  110],
          [ 105,   54,    0,  104,   67],
          [  25,  876,    0,  211,  595],
          [  94,    0,   29,   55,   79],
          [ 159,    3,   78,  992,  460],
          [  34,   95,   43,    6,   19],
          [ 105,  378,  917,    2,    0],
          [ 105,   12,  152,    6,  566],
          [  36,   79,    5,   19,   31],
          [  34,   25,   48,  602,  109],
          [  18,    0,  149,   41,  5

In [117]:
class EmbeddingModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(EmbeddingModel, self).__init__()
    self.target_embedding = Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [202]:
embedding_dim = 32

model = EmbeddingModel(vocab_size, embedding_dim)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(dataset, epochs=100, callbacks=[tensorboard_callback])

Epoch 1/100
9/9 [==============================] - 1s 10ms/step - loss: 1.6093 - accuracy: 0.2465
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 1.6055 - accuracy: 0.5139
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 1.6019 - accuracy: 0.7257
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 1.5977 - accuracy: 0.8715
Epoch 5/100
9/9 [==============================] - 0s 5ms/step - loss: 1.5925 - accuracy: 0.9410
Epoch 6/100
9/9 [==============================] - 0s 5ms/step - loss: 1.5859 - accuracy: 0.9583
Epoch 7/100
9/9 [==============================] - 0s 5ms/step - loss: 1.5774 - accuracy: 0.9688
Epoch 8/100
9/9 [==============================] - 0s 5ms/step - loss: 1.5667 - accuracy: 0.9826
Epoch 9/100
9/9 [==============================] - 0s 5ms/step - loss: 1.5532 - accuracy: 0.9826
Epoch 10/100
9/9 [==============================] - 0s 6ms/step - loss: 1.5365 - accuracy: 0.9826
Epoch 11/100
9/9 [==========

In [208]:
model.summary()

Model: "embedding_model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 w2v_embedding (Embedding)   multiple                  34496     
                                                                 
 embedding_16 (Embedding)    multiple                  34496     
                                                                 
Total params: 68,992
Trainable params: 68,992
Non-trainable params: 0
_________________________________________________________________


In [169]:
%tensorboard --logdir logs

In [205]:
weights = model.get_layer('w2v_embedding').get_weights()[0]
vocab = list(tokenizer_dict.keys())

In [207]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  # out_v.write(word + ',')
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

# TODO
* <s>Load stopwords from file</s>
* Remove stopwords from corpus and vocab
* Use entire law corpus
* Improve BPE performance
* Use Wikipedia corpus
* Create embedder from weights